<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-03-21 08:18:32--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  15.4MB/s    in 2m 9s   

2022-03-21 08:20:43 (10.7 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)
    print(wv_embeddings[word])

float32 (200,)
[ 0.6851772  -1.2778991  -0.41913974  1.3623164  -3.1675398   0.09950767
  0.6402681  -1.1245339  -0.6699619  -0.6998852   0.4936771  -0.40500194
 -3.0706816  -2.2809966   0.85798043  2.7093108   0.3492745  -0.03494101
 -0.22330493  1.2290467   1.7755157  -3.158358   -0.6515983   0.7224096
  2.3193083  -1.7969862   0.40903398 -2.744604   -1.7179952  -0.914309
 -0.75887376 -0.35140672 -0.5182776  -1.9097351  -0.8300773   0.02147918
  1.1783471   0.03169126 -0.3069023   1.6666299   0.6711357  -2.1706133
 -0.11800487  0.22336982 -1.2075394  -0.86297905 -0.63865614  1.1733794
  0.10022762  0.7017279   2.7290728  -0.4640484  -2.1719306  -0.3562852
 -1.8449957   0.10270727  1.1125596  -0.8364318   1.9513408  -0.97937447
  1.2650859   0.06809282  0.6477318  -0.52431005 -0.6103959  -2.979829
 -0.7889965  -0.11004248  1.7603841  -1.0547444  -0.98998    -1.1834062
 -2.5359967  -0.35286787  0.7733574   0.16300043  0.76991326  1.9223119
 -1.2843752  -1.5023688   0.15765315 -2.415087

In [ ]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
import spacy
import numpy as np
from numpy.linalg import norm
from tqdm.notebook import tqdm
import pandas as pd
from gensim.models import Word2Vec

In [ ]:
nlp = spacy.load('en')

In [ ]:
def cosine_sim(v1, v2):
    # v1, v2 (1 x dim)
    return np.array(v1 @ v2.T / norm(v1) / norm(v2))

def find_top5_similar_words(word, embeddings):
    vector_target = embeddings[word]
    similarity = []
    indexes = []
    for w in tqdm(embeddings.index2word):
        w_emb = embeddings[w]
        similarity.append(cosine_sim(w_emb, vector_target))
        indexes.append(w)
    data = pd.DataFrame(similarity, index=indexes, columns=['similarity']).sort_values(by='similarity', ascending=False)
    return data

In [ ]:
df = find_top5_similar_words('dog', wv_embeddings)

  0%|          | 0/1787145 [00:00<?, ?it/s]

В топ-5 наиболее схожих слов входит слово `cats`. В единственном числе слово в список похожих в не выходит.

In [ ]:
df.head(5)

,similarity
dog,1.000000
animal,0.856418
dogs,0.788087
mammal,0.762380
cats,0.762125


In [ ]:
wv_embeddings.most_similar('dog')

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398118972778),
 ('bird', 0.7315489053726196),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349938392639),
 ('labrador', 0.7209131717681885)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
from nltk.tokenize import WordPunctTokenizer

In [ ]:
tokenizer = WordPunctTokenizer()

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

### question_to_vec

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        return: векторное представление для вопроса
    """
    # question = nlp(question)
    # question = ' '.join([token.lemma_ for token in question])
    # print(question)
    question = tokenizer.tokenize(question) #
    
    question_vectors = []
    for word in question:
        if word in embeddings:
            question_vectors.append(embeddings[word])
        else:
            question_vectors.append(np.zeros(dim))
    if question_vectors == []:
        return np.zeros(dim)
    question_vectors = np.array(question_vectors).mean(axis=0)
    return question_vectors

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
test_sent = question_to_vec('I love neural networks', wv_embeddings, tokenizer)
round(test_sent[2], 2)

-0.96

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

In [ ]:
1

1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

In [ ]:
hits_at_1 = 0/1
hits_at_4 = 1/1

print('Hits at 1:', hits_at_1)
print('Hits at 2:', hits_at_4)

Hits at 1: 0.0
Hits at 2: 1.0


In [ ]:
dcg_at_1 = 1/np.log2(1+2) * 0
dcg_at_4 = 1/np.log2(1+2) * 1

print('DCG at 1:', dcg_at_1)
print('DCG at 4:', round(dcg_at_4, 2))

DCG at 1: 0.0
DCG at 4: 0.63


#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
dcg_at_10 = 1/np.log2(1+9) * 1
print('DCG at 10:', round(dcg_at_10, 4))

DCG at 10: 0.301


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(ranks, n):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    ranks = np.asarray(ranks).reshape(-1)
    return np.sum(ranks <= n) / len(ranks)

In [ ]:
def dcg_score(ranks, n):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    ranks = np.asarray(ranks).reshape(-1)
    return (np.sum(1/np.log2(1+(ranks)))/ len(ranks) * np.sum(ranks <= n)) / len(ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split("\t"))
    # data = [c for i in data for c in i]
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

### rank_candidates

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vector = question_to_vec(question, embeddings, tokenizer, dim)
    candidates_positions = {i:n for n,i in enumerate(candidates)}
    candidates_vectors = []
    for candidate in candidates:
        candidates_vectors.append(question_to_vec(candidate, embeddings, tokenizer, dim))
    candidates_vectors = np.asarray(candidates_vectors)
    cos_sims = cosine_similarity(question_vector.reshape(1, -1), candidates_vectors).tolist()[0]
    
    result = list(zip(cos_sims, candidates))
    result = sorted(result, key=lambda x: x[0], reverse=True)
    result = [(candidates_positions[c], c) for _, c in result]
    
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [('*', 'Getting all list items of an unordered list in PHP'), #скрыт
            ('*', 'select2 not displaying search results'), #скрыт
            ('*', 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

In [ ]:
ranks

[(1, 'WPF- How to update the changes in list item of a list'),
 (0, 'Getting all list items of an unordered list in PHP'),
 (2, 'select2 not displaying search results')]

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    try:
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    except Exception:
        pass

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.128 | Hits@   1: 0.286
DCG@   5: 0.177 | Hits@   5: 0.397
DCG@  10: 0.202 | Hits@  10: 0.453
DCG@ 100: 0.304 | Hits@ 100: 0.681
DCG@ 500: 0.394 | Hits@ 500: 0.883
DCG@1000: 0.446 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
words = []
for sents in tqdm(train_data):
    temp = []
    for s in sents:
        # s = s.lower()
        temp.append(tokenizer.tokenize(s))
    words.append([c for i in temp for c in i])

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=300,                 # embedding vector size
                 min_count=5,
                 window=5,
                 workers=4
                 ).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer) # 
    try:
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    except Exception as e:
        pass

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.062 | Hits@   1: 0.181
DCG@   5: 0.096 | Hits@   5: 0.279
DCG@  10: 0.114 | Hits@  10: 0.330
DCG@ 100: 0.191 | Hits@ 100: 0.553
DCG@ 500: 0.279 | Hits@ 500: 0.809
DCG@1000: 0.344 | Hits@1000: 1.000


In [ ]:
# попробуем добиться лучших показателей с лемматизацией с помощью ntlk

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def question_to_vec(question, embeddings, tokenizer, dim=300):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        return: векторное представление для вопроса
    """
    question = tokenizer(question) #
    
    question_vectors = []
    for word in question:
        if word in embeddings:
            question_vectors.append(embeddings[word])
        else:
            question_vectors.append(np.zeros(dim))
    if question_vectors == []:
        return np.zeros(dim)
    question_vectors = np.array(question_vectors).mean(axis=0)
    return question_vectors

wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')

def nltk_process(text):
    nltk_tokenList = word_tokenize(text)
    wordnet_lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in nltk_tokenList:
        nltk_lemmaList.append(wordnet_lemmatizer.lemmatize(word))
    
    return nltk_lemmaList

# words = [[nltk_process(w)] for sent in tqdm(train_data) for w in sent]

words = []
for sents in tqdm(train_data):
    temp = []
    for s in sents:
        s = s.lower()
        temp.append(nltk_process(s))
    words.append([c for i in temp for c in i])

embeddings_trained = Word2Vec(words, # data for model to train on
                 size=300,                 # embedding vector size
                 min_count=5,
                 window=5,
                 workers=4
                 ).wv

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    line = [i.lower() for i in line]
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, nltk_process) # tokenizer
    try:
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    except Exception as e:
        pass
    
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.145 | Hits@   1: 0.304
DCG@   5: 0.213 | Hits@   5: 0.446
DCG@  10: 0.239 | Hits@  10: 0.500
DCG@ 100: 0.361 | Hits@ 100: 0.756
DCG@ 500: 0.450 | Hits@ 500: 0.942
DCG@1000: 0.478 | Hits@1000: 1.000


In [ ]:
# Попробуем улучшить качество с помощью биграмм

from gensim.models import Phrases, phrases
from gensim.models.phrases import Phraser

words = [w.lower().split() for words in tqdm(train_data) for w in words]

bigrams = phrases.Phrases(words)

embeddings_trained = Word2Vec(bigrams[words], # data for model to train on
                 size=300,                 # embedding vector size
                 min_count=5,
                 window=5,
                 workers=4
                 ).wv

  0%|          | 0/1000000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    line = [i.lower() for i in line]
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, nltk_process) # tokenizer
    try:
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    except Exception as e:
        pass

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.096 | Hits@   1: 0.234
DCG@   5: 0.147 | Hits@   5: 0.358
DCG@  10: 0.174 | Hits@  10: 0.424
DCG@ 100: 0.283 | Hits@ 100: 0.690
DCG@ 500: 0.370 | Hits@ 500: 0.901
DCG@1000: 0.411 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
1. Какой принцип токенизации даёт качество лучше и почему?
2. Помогает ли нормализация слов?
    * да, после лемматизации `DCG` и `Hits` немного выросли. 
3. Какие эмбеддинги лучше справляются с задачей и почему?
4. Почему получилось плохое качество решения задачи?
5. Предложите свой подход к решению задачи.